# Generating Code _from_ Your Tests

It's somewhat typical to want to generate unit tests from code, but the _test-driven development_ approach writes the test _first_, then the code. So, let's try that.

## Setting Up

In this notebook, we will use both Ollama running locally and a Granite Code model staged in Replicate.

### Install dependencies

Granite Kitchen comes with a bundle of dependencies that are required for Granite Cookbook recipes. See the list of packages in its [`setup.py`](https://github.com/ibm-granite-community/granite-kitchen/blob/main/setup.py). 

In [1]:
!pip install git+https://github.com/ibm-granite-community/granite-kitchen

  Cloning https://github.com/ibm-granite-community/granite-kitchen to /private/var/folders/66/rbkl_n1s7sn38mx0sg8p1n600000gn/T/pip-req-build-ltwu8tss
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/granite-kitchen /private/var/folders/66/rbkl_n1s7sn38mx0sg8p1n600000gn/T/pip-req-build-ltwu8tss
  Resolved https://github.com/ibm-granite-community/granite-kitchen to commit cee1513c77429d7ddbf0e5a49b29b7bc9ca0d996
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/ibm-granite-community/utils to /private/var/folders/66/rbkl_n1s7sn38mx0sg8p1n600000gn/T/pip-install-8g1yxsxw/ibm-granite-community_22e6602c9d934055a2c348e4259ebdc5
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /private/var/folders/66/rbkl_n1s7sn38mx0sg8p1n600000gn/T/pip-install-8g1yxsxw/ibm-granite-community_22e6602c9d934055a2c348e4259ebdc5
  Resolved https://github.com/ibm-granite-community/utils to commit f1

## Generate the Code from a Hypothesis _Property-Based_ Test Suite

The next two cells install a testing library called `hypothesis` and then read a file of tests for a non-existent `Rational` class (for rational numbers). 

In [4]:
!pip install 'hypothesis[cli]'

Load our Hypothesis tests as a string.
This test file is adapted from this GitHub project: 

https://github.com/deanwampler/tdd-hypothesis-example

In [5]:
with open("test_rational.py") as f:
    tests = f.read()
print(tests)

# Example unit tests using Hypothesis for property-based testing.
# Adapted from: https://github.com/deanwampler/tdd-hypothesis-example
# Hypothesis website: https://hypothesis.readthedocs.io/en/latest/

from hypothesis import given, strategies as st
import unittest
from rational import Rational
from math import gcd

class TestRational(unittest.TestCase):
    """
    Test the features implemented curently by Rational.
    Add new tests for Rational arithmetic operations, like multiplication and addition,
    watch the test fail, then implement the feature and ensure the test now passes.
    See also other properties described in the Rational Wikipedia page:
    https://en.wikipedia.org/wiki/Rational_number
    
    Also, try adding a second way to construct Rationals that accepts a string
    argument, "M/N". (Now you really have to think about handling input errors!) 
    What are the requirements for valid strings, e.g., for "M" and "N"?
    If an invalid string is provided, how shou

## Try Generating an Implementation of `Rational`

Let's try Ollama locally and also calling the model in Replicate. The local model is quantized, so we expect it not to work as well.

In [6]:
!pip install ollama

In [7]:
import ollama

In [12]:
default_system_prompt = f"""
    You are a helpful software engineer. You write clear, concise, well-commented code. 
    Make sure you only generate python code that makes the Hypothesis tests pass!
    """

In [17]:
def ollama_code_gen(test_code: str,
                    system_prompt:str = default_system_prompt,
                    examples: str = "",
                    model: str ='granite-code:20b') -> str:
    user_prompt = f"""
        {examples}
        Analyze the following Python tests written with the hypothesis library
        (https://hypothesis.readthedocs.io/en/latest/). Write a definition of the Rational class required
        to make these tests pass. DO NOT echo the tests in your output! Here is the test code:
        {test_code}
        Answer:"""

    response = ollama.chat(model=model, messages=[
      {
        'role':'system',
        'content': system_prompt
      },
      {
        'role': 'user',
        'content': user_prompt
      },
    ])

    result = response['message']['content']
    print(result)
    return result

In [18]:
# Construct a prompt with no tabs and newlines.
ollama_result = ollama_code_gen(tests)

Here is an implementation of the Rational class that makes these tests pass:

```python
from math import gcd

class Rational:
    """
    A class representing rational numbers.

    Attributes:
        numerator (int): The numerator of the rational number.
        denominator (int): The denominator of the rational number.

    Methods:
        __init__(self, numerator, denominator): Initializes a Rational object with the given numerator and denominator.
        __str__(self): Returns a string representation of the Rational object.
        __eq__(self, other): Checks if two Rational objects are equal.
    """

    def __init__(self, numerator, denominator):
        """
        Initializes a Rational object with the given numerator and denominator.

        Args:
            numerator (int): The numerator of the rational number.
            denominator (int): The denominator of the rational number.

        Raises:
            ValueError: If the denominator is zero.
        """
        i

### Let's Try It!

Copy the generated `Rational` class definition in the output above. (If there isn't one, the generation failed, so try running it again!!) Paste that code into the `rational/rational.py` file, which currently contains just comments.

Don't include the test code or any markdown or other text included in the generated output!

Do include appropriate `import` statements, e.g., you may see the generated code calls `gcd` (for
_greatest common divisor_, which would require this import `from math import gcd`.

Is the indentation correct? Make sure each level is properly indented.

Then, run the following cell, which will run the test code we read above.

In [19]:
!python ./test_rational.py

......
----------------------------------------------------------------------
Ran 6 tests in 0.189s

OK


Did the tests pass? If not, what can you change in the generated `Rational` code to make them pass. If the test did pass, when you look at the generated code, does it look correct? Compare what it does with the definition of Rational you find at some reliable source, like [this Wikipedia page](https://en.wikipedia.org/wiki/Rational_number).

Also, compare what you got above to `rational/rational-good-example.py`, which was generated by one of our test runs. (It is not the only possible "good" definition...)

Let's try invoking the model staged in Replicate.

> **NOTE:** You can select a Granite Code model from the [`ibm-granite`](https://replicate.com/ibm-granite) org on Replicate. Here we use the Replicate Langchain client to connect to the model.
>
> 
To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [21]:
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var

default_replicate_model = Replicate(
    model="ibm-granite/granite-20b-code-instruct-8k",
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
)

In [22]:
def replicate_code_gen(test_code: str,
                       system_prompt:str = default_system_prompt,
                       examples: str = "",
                       replicate_model: Replicate = default_replicate_model) -> str:
    prompt = f"""
        system prompt: {system_prompt}
        examples: {examples}
        Analyze the following Python tests written with the hypothesis library
        (https://hypothesis.readthedocs.io/en/latest/). Write a definition of the Rational class required
        to make these tests pass. DO NOT echo the tests in your output! Here is the test code:
        {test_code}
        Answer:"""

    result = replicate_model.invoke(prompt)

#    result = response['message']['content']
    print(result)
    return result

In [23]:
replicate_result = replicate_code_gen(tests)

Here is the definition of the Rational class that makes the Hypothesis tests pass:

```python
class Rational:
 def __init__(self, numerator, denominator):
 self.numerator = numerator
 self.denominator = denominator

 def __eq__(self, other):
 return self.numerator * other.denominator == self.denominator * other.numerator
```

The Rational class has two instance variables, numerator and denominator, which are initialized in the constructor. The __eq__ method is overridden to compare two instances of Rational based on the rule that a/b == c/d if and only if ad == bc. This ensures that the Hypothesis tests pass. 


As above, copy just the `Rational` definition to `rational/rational.py` and run the tests:

In [25]:
!python ./test_rational.py

...F.F
FAIL: test_init_takes_numerator_denominator (__main__.TestRational.test_init_takes_numerator_denominator)
A "relatively-trivial" test, but note that the returned
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/deanwampler/ibm/ibm-granite-community/granite-code-cookbook/recipes/Code_Gen_from_Tests/./test_rational.py", line 29, in test_init_takes_numerator_denominator
    def test_init_takes_numerator_denominator(self, numer, denom):
               ^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/granite-311b/lib/python3.11/site-packages/hypothesis/core.py", line 1722, in wrapped_test
    raise the_error_hypothesis_found
  File "/Users/deanwampler/ibm/ibm-granite-community/granite-code-cookbook/recipes/Code_Gen_from_Tests/./test_rational.py", line 38, in test_init_takes_numerator_denominator
    self.assertEqual(denom // divisor, rat.denominator)
AssertionError: 1 != 2
Falsifying example: test_init_

## For Further Study
* Add new tests to `test_rational.py` for other operators, like `*`, `/`, `+`, `-`, `<`, `<=`, `>`, `>=`, then see if these operators are properly implemented. Keep in mind that `Rational(i*numerator, i*denominator)`, for some integer `i`, numerator `numerator`, and denominator `denominator`, is always "rationalized" to `Rational(numerator, denominator)`.